In [2]:
import pickle
import pandas as pd
from rdkit import Chem
from rhea.chem.molecule import Molecule
from rdkit.Chem import rdDetermineBonds
import py3Dmol
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True

def draw_with_spheres(mol):
    v = py3Dmol.view(width=300,height=300)
    IPythonConsole.addMolToView(mol,v)
    v.zoomTo()
    v.setStyle({'sphere':{'radius':0.3},'stick':{'radius':0.2}});
    v.show()

In [ ]:
#https://greglandrum.github.io/rdkit-blog/posts/2022-12-18-introducing-rdDetermineBonds.html

In [2]:
df = pd.read_pickle('../data/QM9/raw/gdb9.pkl')

In [3]:
m = df.ROMol.sample(1).values[0]
m = Molecule(m)

In [4]:
draw_with_spheres(m.molecule)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
m.get_coordinates()

array([[ 0.0098    ,  1.48769999, -0.1013    ],
       [ 0.0257    ,  0.0279    , -0.0317    ],
       [-1.14549994, -0.67540002,  0.63080001],
       [-0.73030001, -2.12280011,  0.39289999],
       [ 0.5711    , -2.1013999 ,  0.70770001],
       [ 0.787     , -0.63950002,  1.09850001],
       [-0.45140001, -0.47060001,  2.04959989],
       [-0.53100002,  0.78500003,  2.66989994]])

In [6]:
block = "{}\n\n".format(len(m.get_atoms()))
count = 0
for at, coord in zip(m.get_atoms(), m.get_coordinates()):
    at = at.GetSymbol()
    x, y, z = coord
    block += f"{at} {x:.4f} {y:.4f} {z:.4f}\n"

print(block)

8

C 0.0098 1.4877 -0.1013
N 0.0257 0.0279 -0.0317
C -1.1455 -0.6754 0.6308
C -0.7303 -2.1228 0.3929
C 0.5711 -2.1014 0.7077
C 0.7870 -0.6395 1.0985
C -0.4514 -0.4706 2.0496
O -0.5310 0.7850 2.6699



In [12]:
raw_mol = Chem.MolFromXYZBlock(block)

In [13]:
conn_mol = Chem.Mol(raw_mol)
rdDetermineBonds.DetermineBonds(conn_mol,charge=0)
draw_with_spheres(conn_mol)

ValueError: Final molecular charge does not match input; could not find valid bond ordering

In [8]:
draw_with_spheres(raw_mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [9]:
conn_mol = Chem.Mol(raw_mol)
rdDetermineBonds.DetermineConnectivity(conn_mol)

In [10]:
draw_with_spheres(conn_mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
rdDetermineBonds.DetermineBondOrders(conn_mol, charge=0)
draw_with_spheres(conn_mol)

ValueError: Final molecular charge does not match input; could not find valid bond ordering

In [6]:
molecule = """<LIGAND>
Cn1c(=O)c2c(ncn2C)n(C)c1=O
<XYZ>
C 0.4029 1.4723 -0.1406
N 1.3644 0.1596 -0.3010
C -1.1981 0.3139 0.3332
N 2.0035 -1.8714 0.0184
N 1.1810 1.4370 1.3258
C -1.3547 -0.1193 -0.8744
C 1.8995 -0.3421 -0.0087
O -2.2328 0.2256 -0.6995
C 1.6100 -0.6686 1.5922
O -1.1704 -0.0924 -0.0939"""

smiles, xyz = molecule.split("<XYZ>")
smiles = smiles.split("<LIGAND>")[1].strip()
xyz = xyz.strip()

mol = Chem.MolFromSmiles(smiles, sanitize=True)   # 2D molecule

length = mol.GetNumAtoms()
coords = xyz.split("\n")[:length]
conn = xyz.split("\n")[length:]

block = "{}\n\n".format(length)
print(length)
for i in range(length):
    print(coords[i])
    at, x, y, z = coords[i].split()
    x, y, z = float(x), float(y), float(z)
    block += f"{at} {x:.4f} {y:.4f} {z:.4f}\n"

14
C 0.4029 1.4723 -0.1406
N 1.3644 0.1596 -0.3010
C -1.1981 0.3139 0.3332
N 2.0035 -1.8714 0.0184
N 1.1810 1.4370 1.3258
C -1.3547 -0.1193 -0.8744
C 1.8995 -0.3421 -0.0087
O -2.2328 0.2256 -0.6995
C 1.6100 -0.6686 1.5922
O -1.1704 -0.0924 -0.0939


IndexError: list index out of range